In [1]:
import keras
import datetime
import numpy as np
import pandas as pd
from   src.Src import calcular_pesos_classes

from sklearn.metrics         import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing   import StandardScaler, LabelEncoder

2024-04-23 17:59:03.828677: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-23 17:59:03.869215: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-23 17:59:03.869285: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-23 17:59:03.870488: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-23 17:59:03.876542: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-23 17:59:03.877016: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
data = pd.read_csv('dataset/apple_quality.csv')

In [3]:
data.drop(index=data.loc[(data.isna()==True).any(axis=1)==True].index,columns=['A_id'] ,inplace=True) 

In [4]:
data.head(3)

,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590483,good
1,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809367,good
2,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636473,bad


In [5]:
data.describe()

,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000
mean,-0.503015,-0.989547,-0.470479,0.985478,0.512118,0.498277
std,1.928059,1.602507,1.943441,1.402757,1.930286,1.874427
min,-7.151703,-7.149848,-6.894485,-6.055058,-5.961897,-5.864599
25%,-1.816765,-2.011770,-1.738425,0.062764,-0.801286,-0.771677
50%,-0.513703,-0.984736,-0.504758,0.998249,0.534219,0.503445
75%,0.805526,0.030976,0.801922,1.894234,1.835976,1.766212
max,6.406367,5.790714,6.374916,7.619852,7.364403,7.237837


# Keras

In [6]:
data['Acidity'] = data['Acidity'].astype(np.float64)

In [7]:
X = data.drop(columns=['Quality']).values
y = data['Quality'].values

In [8]:
X = StandardScaler().fit_transform(X)
y = LabelEncoder().fit_transform(y)

In [9]:
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=42)

In [10]:
xtrain.shape, xtest.shape

((3000, 7), (1000, 7))

In [11]:
def ModelBuild():
    pass

%load_ext tensorboard

In [12]:
input_  = keras.layers.Input(shape=xtrain.shape[1:])
oculta1 = keras.layers.Dense(units=14, activation='relu')(input_)
oculta2 = keras.layers.Dense(units=40, activation='relu')(oculta1)
oculta3 = keras.layers.Dense(units=20, activation='relu')(oculta2)
output  = keras.layers.Dense(units=1, activation='relu')(oculta3)

model = keras.Model(inputs=[input_], outputs=[output])

model.compile(
    loss      = keras.losses.binary_crossentropy,
    optimizer = keras.optimizers.Adam(),
    metrics   = [keras.metrics.binary_accuracy]
)

In [13]:
pesos = calcular_pesos_classes(ytrain)
pesos

{0: 0.9993337774816788, 1: 1.0006671114076051}

In [14]:
logsFit = 'logs/fit/'+datetime.datetime.now().strftime("%Y_%m_%d - %H_%M_%S")
tensorBoard = keras.callbacks.TensorBoard(log_dir=logsFit, histogram_freq=1)

In [15]:
history = model.fit( xtrain, ytrain, epochs=20, class_weight=pesos , callbacks=[tensorBoard])

Epoch 1/20


94/94 [==============================] - 2s 6ms/step - loss: 0.8911 - binary_accuracy: 0.6377
Epoch 2/20
94/94 [==============================] - 0s 4ms/step - loss: 0.5772 - binary_accuracy: 0.7530
Epoch 3/20
94/94 [==============================] - 0s 5ms/step - loss: 0.4543 - binary_accuracy: 0.7923
Epoch 4/20
94/94 [==============================] - 0s 4ms/step - loss: 0.4057 - binary_accuracy: 0.8213
Epoch 5/20
94/94 [==============================] - 0s 4ms/step - loss: 0.3961 - binary_accuracy: 0.8297
Epoch 6/20
94/94 [==============================] - 0s 5ms/step - loss: 0.4463 - binary_accuracy: 0.8180
Epoch 7/20
94/94 [==============================] - 0s 4ms/step - loss: 0.4116 - binary_accuracy: 0.8440
Epoch 8/20
94/94 [==============================] - 0s 5ms/step - loss: 0.3683 - binary_accuracy: 0.8450
Epoch 9/20
94/94 [==============================] - 0s 4ms/step - loss: 0.3402 - binary_accuracy: 0.8577
Epoch 10/20
94/94 [==============================] - 0s 5ms/step -

In [16]:
pred = model.predict(xtest)>0.5

32/32 [==============================] - 0s 2ms/step


In [17]:
accuracy_score(ytest,pred)

0.899

In [18]:
print(classification_report(ytest, pred))

              precision    recall  f1-score   support

           0       0.88      0.92      0.90       495
           1       0.92      0.88      0.90       505

    accuracy                           0.90      1000
   macro avg       0.90      0.90      0.90      1000
weighted avg       0.90      0.90      0.90      1000



In [19]:
logsFit

'logs/fit/2024_04_23 - 17_59_07'

In [20]:
%tensorboard --logdir='logs/fit'

In [21]:
from tensorboard import notebook
notebook.list()

Known TensorBoard instances:
  - port 6011: logdir logs (started 0:07:56 ago; pid 11288)
  - port 6012: logdir logs/fit (started 0:06:19 ago; pid 11474)
  - port 6013: logdir logs (started 0:05:15 ago; pid 11627)
  - port 6014: logdir logs/fit (started 0:04:47 ago; pid 11733)
  - port 6017: logdir logs/fit (started 0:00:01 ago; pid 13297)
